<a href="https://colab.research.google.com/github/aderdouri/PINNs/blob/master/Tutorials/IPINN_Navier_Stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import scipy.io
import torch
import io

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
def load_training_data(num):
    # Changed to raw URL
    url = "https://github.com/maziarraissi/PINNs/raw/master/main/Data/cylinder_nektar_wake.mat"
    # Step 2: Download the file content
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    # Step 3: Load the .mat file into memory
    data = scipy.io.loadmat(io.BytesIO(response.content))

    U_star = data["U_star"]  # N x 2 x T
    P_star = data["p_star"]  # N x T
    t_star = data["t"]  # T x 1
    X_star = data["X_star"]  # N x 2
    N = X_star.shape[0]
    T = t_star.shape[0]
    # Rearrange Data
    XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
    YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
    TT = np.tile(t_star, (1, N)).T  # N x T
    UU = U_star[:, 0, :]  # N x T
    VV = U_star[:, 1, :]  # N x T
    PP = P_star  # N x T
    x = XX.flatten()[:, None]  # NT x 1
    y = YY.flatten()[:, None]  # NT x 1
    t = TT.flatten()[:, None]  # NT x 1
    u = UU.flatten()[:, None]  # NT x 1
    v = VV.flatten()[:, None]  # NT x 1
    p = PP.flatten()[:, None]  # NT x 1
    # training domain: X × Y = [1, 8] × [−2, 2] and T = [0, 7]
    data1 = np.concatenate([x, y, t, u, v, p], 1)
    data2 = data1[:, :][data1[:, 2] <= 7]
    data3 = data2[:, :][data2[:, 0] >= 1]
    data4 = data3[:, :][data3[:, 0] <= 8]
    data5 = data4[:, :][data4[:, 1] >= -2]
    data_domain = data5[:, :][data5[:, 1] <= 2]
    # choose number of training points: num =7000
    idx = np.random.choice(data_domain.shape[0], num, replace=False)
    x_train = data_domain[idx, 0:1]
    y_train = data_domain[idx, 1:2]
    t_train = data_domain[idx, 2:3]
    u_train = data_domain[idx, 3:4]
    v_train = data_domain[idx, 4:5]
    p_train = data_domain[idx, 5:6]
    return [x_train, y_train, t_train, u_train, v_train, p_train]

In [4]:
# Loss function
def loss_function(x, y, t, u_obs, v_obs, p_obs):
    C1 = torch.exp(log_C1)
    C2 = torch.exp(log_C2)

    # Ensure gradients can be computed
    t.requires_grad_(True)
    x.requires_grad_(True)
    y.requires_grad_(True)

    # Forward pass
    output = model(x, y, t)
    u_pred, v_pred, p_pred = output[:, 0:1], output[:, 1:2], output[:, 2:3]

    # Compute derivatives
    u_t = gradients(u_pred, t)
    u_x = gradients(u_pred, x)
    u_y = gradients(u_pred, y)
    u_xx = gradients(u_x, x)
    u_yy = gradients(u_y, y)

    v_t = gradients(v_pred, t)
    v_x = gradients(v_pred, x)
    v_y = gradients(v_pred, y)
    v_xx = gradients(v_x, x)
    v_yy = gradients(v_y, y)

    p_x = gradients(p_pred, x)
    p_y = gradients(p_pred, y)

    # Residuals of Navier-Stokes equations
    f_u = u_t + C1 * (u_pred * u_x + v_pred * u_y) + p_x - C2 * (u_xx + u_yy)
    f_v = v_t + C1 * (u_pred * v_x + v_pred * v_y) + p_y - C2 * (v_xx + v_yy)

    # Continuity equation
    f_cont = gradients(u_pred, x) + gradients(v_pred, y)

    # Physics-based loss
    physics_loss = torch.mean(f_u**2) + torch.mean(f_v**2) + torch.mean(f_cont**2)

    # Data loss (mean squared error with observed data)
    data_loss = torch.mean((u_pred - u_obs)**2) + \
                torch.mean((v_pred - v_obs)**2) + \
                torch.mean((p_pred - p_obs)**2)

    # Total loss
    total_loss = physics_loss + data_loss
    return total_loss

In [5]:
# Helper function for gradient computation
def gradients(y, x):
    return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), create_graph=True, retain_graph=True)[0]

# Define the PINN model
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden_layers = nn.Sequential(
            nn.Linear(3, 50),  # Input: (x, y, t)
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 3)   # Output: (v_x, v_y, p)
        )

    def forward(self, t, x, y):
        inputs = torch.cat([t, x, y], dim=1)
        return self.hidden_layers(inputs)

# Define the unknown parameters (C1 and C2)
log_C1 = torch.tensor([0.0], requires_grad=True, dtype=torch.float32)  # log(C1) for positivity
log_C2 = torch.tensor([0.0], requires_grad=True, dtype=torch.float32)  # log(C2) for positivity

# Initialize the model
model = PINN()

In [ ]:
# Optimizer for both model parameters and unknown constants
optimizer = optim.Adam(list(model.parameters()) + [log_C1, log_C2], lr=0.001)

# Training data (replace with actual data)
#n_points = 1000
#x_train = torch.rand(n_points, 1, requires_grad=True)
#y_train = torch.rand(n_points, 1, requires_grad=True)
#t_train = torch.rand(n_points, 1, requires_grad=True)
#v_train = torch.cat([torch.sin(torch.pi * x_train), torch.cos(torch.pi * y_train)], dim=1)  # Example: true velocities
#p_train = torch.zeros_like(x_train)  # Example: true pressure


# Training data
# Get the training data: num = 7000
[x_train, y_train, t_train, u_train, v_train, p_train] = load_training_data(num=7000)

x_train = torch.tensor(x_train, dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y_train, dtype=torch.float32, requires_grad=True)
t_train = torch.tensor(t_train, dtype=torch.float32, requires_grad=True)
u_train = torch.tensor(u_train, dtype=torch.float32)
v_train = torch.tensor(v_train, dtype=torch.float32)
p_train = torch.tensor(p_train, dtype=torch.float32)

# Training loop
n_epochs = 20000
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = loss_function(x_train, y_train, t_train, u_train, v_train, p_train)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}, C1: {torch.exp(log_C1).item():.6f}, C2: {torch.exp(log_C2).item():.6f}")

print("Training complete!")
print(f"Learned C1: {torch.exp(log_C1).item():.6f}")
print(f"Learned C2: {torch.exp(log_C2).item():.6f}")

Epoch 0, Loss: 0.870026, C1: 1.001001, C2: 0.999000
